In [2]:
from transformers import  AutoModel,AutoTokenizer,AutoModelForMaskedLM , Trainer,TrainingArguments,\
BitsAndBytesConfig,pipeline,default_data_collator,DataCollatorWithPadding,DataCollatorForLanguageModeling
from transformers.utils import move_cache
#from chat_template_utils import get_json_schema
#from llama_cpp import Llama
from utils import *
from peft import *
import datasets
#import torchmetrics
import torch
from torch.utils.data.dataloader import DataLoader,Dataset
#from hqq.engine.hf import HQQModelForCausalLM
#from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
#import deepspeed
import os
import pandas as pd
import json
#from var_dump import var_dump
from bs4 import BeautifulSoup,PageElement,Comment
import matplotlib.pyplot as plt
import seaborn as sns

cache_dir='/var/tmp/.cache/' #'/proj/ciptmp/ix05ogym/.cache/'
output_dir = cache_dir+'outputs/'

move_cache(cache_dir)

def pd_full_screen():
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    
def pd_normal_screen():
    pd.set_option('display.max_rows', 10)
    pd.set_option('display.max_columns', 5)


No ROCm runtime is found, using ROCM_HOME='/usr'
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


0it [00:00, ?it/s]

In [ ]:
train_dataset_2 = pd.read_excel('query_element_7.xlsx')
#train_dataset_2 = train_dataset_2.dropna(['query','element','label']).drop_duplicates(['query','element','label']).reset_index(drop=True)
c = train_dataset_2["label"] == "latitude"
train_dataset_2.loc[c, "label"] = 'distance'
c = train_dataset_2["label"] == "longitude"
train_dataset_2.loc[c, "label"] = 'distance'
c = train_dataset_2["label"] == "english language level"
train_dataset_2.loc[c, "label"] = 'English language level'
c = train_dataset_2["label"] == "job"
train_dataset_2.loc[c, "label"] = 'job title'
c = train_dataset_2["label"] == "title"
train_dataset_2.loc[c, "label"] = 'person title'
train_dataset_2

In [ ]:
c = train_dataset_2['element'].str.contains('type="radio"')
h=train_dataset_2.loc[c,:].reset_index(drop=True)
#h = BeautifulSoup(h[0],'html.parser').find_all('option')
#h.attrs.get('value')
for i,x in h.iterrows():
    print(x['query'])
    print(x['element'])
    print(x['label'])
    print('------------------------------------------------')

In [34]:
html = """

<select name="gender">
    <option value="">Select Gender</option>
    <option value="male">Male</option>
    <option value="female">Female</option>
    <option value="non-binary">Non-binary</option>
    <option value="prefer-not-to-say">Prefer not to say</option>
</select>

<select name="language">
    <option value="">Select Language</option>
    <option value="english">English</option>
    <option value="spanish">Spanish</option>
    <option value="french">French</option>
    <option value="chinese">Chinese</option>
    <option value="hindi">Hindi</option>
    <option value="arabic">Arabic</option>
    <option value="other">Other</option>
</select>

<select name="marital-status">
    <option value="">Select Marital Status</option>
    <option value="single">Single</option>
    <option value="married">Married</option>
    <option value="divorced">Divorced</option>
    <option value="widowed">Widowed</option>
</select>

<select name="visa-status">
    <option value="">Select Visa Status</option>
    <option value="tourist">Tourist Visa</option>
    <option value="student">Student Visa</option>
    <option value="work">Work Visa</option>
    <option value="resident">Resident Visa</option>
    <option value="other">Other</option>
</select>

<select name="title">
    <option value="">Select Title</option>
    <option value="mr">Mr.</option>
    <option value="mrs">Mrs.</option>
    <option value="ms">Ms.</option>
    <option value="dr">Dr.</option>
    <option value="prof">Prof.</option>
</select>

<select name="employment-status">
    <option value="">Select Employment Status</option>
    <option value="employed">Employed</option>
    <option value="self-employed">Self-employed</option>
    <option value="unemployed">Unemployed</option>
    <option value="student">Student</option>
    <option value="retired">Retired</option>
</select>

<select name="education-level">
    <option value="">Select Education Level</option>
    <option value="high-school">High School</option>
    <option value="bachelors">Bachelor's Degree</option>
    <option value="masters">Master's Degree</option>
    <option value="phd">Ph.D.</option>
    <option value="other">Other</option>
</select>

<select name="age-group">
    <option value="">Select Age Group</option>
    <option value="under-18">Under 18</option>
    <option value="18-24">18-24</option>
    <option value="25-34">25-34</option>
    <option value="35-44">35-44</option>
    <option value="45-54">45-54</option>
    <option value="55-64">55-64</option>
    <option value="65-plus">65+</option>
</select>

<select name="nationality">
    <option value="">Select Nationality</option>
    <option value="american">American</option>
    <option value="canadian">Canadian</option>
    <option value="british">British</option>
    <option value="australian">Australian</option>
    <option value="indian">Indian</option>
    <option value="chinese">Chinese</option>
    <option value="other">Other</option>
</select>

<select name="income-range">
    <option value="">Select Income Range</option>
    <option value="under-20k">Under $20,000</option>
    <option value="20k-50k">$20,000 - $50,000</option>
    <option value="50k-100k">$50,000 - $100,000</option>
    <option value="100k-200k">$100,000 - $200,000</option>
    <option value="200k-plus">Over $200,000</option>
</select>




"""

html2 = """
<select name="geschlecht">
    <option value="">Geschlecht wählen</option>
    <option value="männlich">Männlich</option>
    <option value="weiblich">Weiblich</option>
    <option value="non-binary">Non-binary</option>
    <option value="keine-angabe">Keine Angabe</option>
</select>

<select name="sprache">
    <option value="">Sprache wählen</option>
    <option value="englisch">Englisch</option>
    <option value="spanisch">Spanisch</option>
    <option value="französisch">Französisch</option>
    <option value="chinesisch">Chinesisch</option>
    <option value="hindi">Hindi</option>
    <option value="arabisch">Arabisch</option>
    <option value="andere">Andere</option>
</select>

<select name="familienstand">
    <option value="">Familienstand wählen</option>
    <option value="ledig">Ledig</option>
    <option value="verheiratet">Verheiratet</option>
    <option value="geschieden">Geschieden</option>
    <option value="verwitwet">Verwitwet</option>
</select>

<select name="visumstatus">
    <option value="">Visumstatus wählen</option>
    <option value="tourist">Touristenvisum</option>
    <option value="student">Studentenvisum</option>
    <option value="arbeit">Arbeitsvisum</option>
    <option value="resident">Aufenthaltsvisum</option>
    <option value="andere">Andere</option>
</select>

<select name="anrede">
    <option value="">Anrede wählen</option>
    <option value="herr">Herr</option>
    <option value="frau">Frau</option>
    <option value="fräulein">Fräulein</option>
    <option value="dr">Dr.</option>
    <option value="prof">Prof.</option>
</select>

<select name="beschäftigungsstatus">
    <option value="">Beschäftigungsstatus wählen</option>
    <option value="angestellt">Angestellt</option>
    <option value="selbstständig">Selbstständig</option>
    <option value="arbeitslos">Arbeitslos</option>
    <option value="student">Student</option>
    <option value="rentner">Rentner</option>
</select>

<select name="bildungsniveau">
    <option value="">Bildungsniveau wählen</option>
    <option value="high-school">Hochschule</option>
    <option value="bachelor">Bachelor</option>
    <option value="master">Master</option>
    <option value="phd">Ph.D.</option>
    <option value="andere">Andere</option>
</select>

<select name="altersgruppe">
    <option value="">Altersgruppe wählen</option>
    <option value="unter-18">Unter 18</option>
    <option value="18-24">18-24</option>
    <option value="25-34">25-34</option>
    <option value="35-44">35-44</option>
    <option value="45-54">45-54</option>
    <option value="55-64">55-64</option>
    <option value="65-plus">65+</option>
</select>

<select name="nationalität">
    <option value="">Nationalität wählen</option>
    <option value="amerikanisch">Amerikanisch</option>
    <option value="kanadisch">Kanadisch</option>
    <option value="britisch">Britisch</option>
    <option value="australisch">Australisch</option>
    <option value="indisch">Indisch</option>
    <option value="chinesisch">Chinesisch</option>
    <option value="andere">Andere</option>
</select>

<select name="einkommensbereich">
    <option value="">Einkommensbereich wählen</option>
    <option value="unter-20k">Unter 20.000 €</option>
    <option value="20k-50k">20.000 € - 50.000 €</option>
    <option value="50k-100k">50.000 € - 100.000 €</option>
    <option value="100k-200k">100.000 € - 200.000 €</option>
    <option value="200k-plus">Über 200.000 €</option>
</select>

"""

html3 = """
<select name="wohnort">
    <option value="">Wohnort wählen</option>
    <option value="stadt">Stadt</option>
    <option value="vorstadt">Vorstadt</option>
    <option value="land">Land</option>
</select>

<select name="transportmittel">
    <option value="">Transportmittel wählen</option>
    <option value="auto">Auto</option>
    <option value="fahrrad">Fahrrad</option>
    <option value="öffentliche-verkehrsmittel">Öffentliche Verkehrsmittel</option>
    <option value="zu-fuß">Zu Fuß</option>
    <option value="motorrad">Motorrad</option>
    <option value="andere">Andere</option>
</select>

<select name="ernährung">
    <option value="">Ernährungsweise wählen</option>
    <option value="fleischesser">Fleischesser</option>
    <option value="vegetarier">Vegetarier</option>
    <option value="veganer">Veganer</option>
    <option value="pescetarier">Pescetarier</option>
    <option value="andere">Andere</option>
</select>

<select name="haustiere">
    <option value="">Haustiere wählen</option>
    <option value="hund">Hund</option>
    <option value="katze">Katze</option>
    <option value="vogel">Vogel</option>
    <option value="fisch">Fisch</option>
    <option value="kein-haustier">Kein Haustier</option>
    <option value="andere">Andere</option>
</select>

<select name="wohnungsart">
    <option value="">Wohnungsart wählen</option>
    <option value="eigentumswohnung">Eigentumswohnung</option>
    <option value="mietwohnung">Mietwohnung</option>
    <option value="einfamilienhaus">Einfamilienhaus</option>
    <option value="mehrfamilienhaus">Mehrfamilienhaus</option>
    <option value="wohnheim">Wohnheim</option>
</select>

<select name="fitnessniveau">
    <option value="">Fitnessniveau wählen</option>
    <option value="anfänger">Anfänger</option>
    <option value="mittel">Mittel</option>
    <option value="fortgeschritten">Fortgeschritten</option>
    <option value="athlet">Athlet</option>
</select>

<select name="musikgeschmack">
    <option value="">Musikgeschmack wählen</option>
    <option value="pop">Pop</option>
    <option value="rock">Rock</option>
    <option value="klassik">Klassik</option>
    <option value="hip-hop">Hip-Hop</option>
    <option value="jazz">Jazz</option>
    <option value="andere">Andere</option>
</select>

<select name="lieblingsjahreszeit">
    <option value="">Lieblingsjahreszeit wählen</option>
    <option value="frühling">Frühling</option>
    <option value="sommer">Sommer</option>
    <option value="herbst">Herbst</option>
    <option value="winter">Winter</option>
</select>

<select name="lieblingsfarbe">
    <option value="">Lieblingsfarbe wählen</option>
    <option value="rot">Rot</option>
    <option value="blau">Blau</option>
    <option value="grün">Grün</option>
    <option value="gelb">Gelb</option>
    <option value="schwarz">Schwarz</option>
    <option value="weiß">Weiß</option>
    <option value="andere">Andere</option>
</select>

<select name="kommunikationsmethode">
    <option value="">Kommunikationsmethode wählen</option>
    <option value="telefon">Telefon</option>
    <option value="email">E-Mail</option>
    <option value="sms">SMS</option>
    <option value="videoanruf">Videoanruf</option>
    <option value="persönlich">Persönlich</option>
</select>
"""

checks="""
<div>
    <label>Geschlecht:</label><br>
    <label for="male">Männlich</label><br>
    <input type="checkbox" id="male" name="gender" value="male">
    <label for="female">Weiblich</label><br>
    <input type="checkbox" id="female" name="gender" value="female">
    <label for="non-binary">Non-binary</label><br>
    
    <input type="checkbox" id="non-binary" name="gender" value="non-binary">
    <label for="prefer-not-to-say">Keine Angabe</label>
    
    <input type="checkbox" id="prefer-not-to-say" name="gender" value="prefer-not-to-say">
</div>

<div>
    <label>Sprache:</label><br>
    <label for="english">Englisch</label><br>
    
    <input type="checkbox" id="english" name="language" value="english">
    <label for="spanish">Spanisch</label><br>
    
    <input type="checkbox" id="spanish" name="language" value="spanish">
    <label for="french">Französisch</label><br>
    
    <input type="checkbox" id="french" name="language" value="french">
    <label for="chinese">Chinesisch</label><br>
    
    <input type="checkbox" id="chinese" name="language" value="chinese">
    <label for="hindi">Hindi</label><br>
    
    <input type="checkbox" id="hindi" name="language" value="hindi">
    <label for="arabic">Arabisch</label><br>
    
    <input type="checkbox" id="arabic" name="language" value="arabic">
    <label for="other-language">Andere</label>
    
    <input type="checkbox" id="other-language" name="language" value="other">
</div>

<div>
    <label>Familienstand:</label><br>
    <label for="single">Ledig</label><br>
    
    <input type="checkbox" id="single" name="marital-status" value="single">
    <label for="married">Verheiratet</label><br>
    
    <input type="checkbox" id="married" name="marital-status" value="married">
    <label for="divorced">Geschieden</label><br>
    
    <input type="checkbox" id="divorced" name="marital-status" value="divorced">
    <label for="widowed">Verwitwet</label>
    
    <input type="checkbox" id="widowed" name="marital-status" value="widowed">
</div>

<div>
    <label>Visumstatus:</label><br>
    <label for="tourist">Touristenvisum</label><br>
    
    <input type="checkbox" id="tourist" name="visa-status" value="tourist">
    <label for="student">Studentenvisum</label><br>
    
    <input type="checkbox" id="student" name="visa-status" value="student">
    <label for="work">Arbeitsvisum</label><br>
    
    <input type="checkbox" id="work" name="visa-status" value="work">
    <label for="resident">Aufenthaltsvisum</label><br>
    
    <input type="checkbox" id="resident" name="visa-status" value="resident">
    <label for="other-visa">Andere</label>
    
    <input type="checkbox" id="other-visa" name="visa-status" value="other">
</div>

<div>
    <label>Anrede:</label><br>
    <label for="mr">Herr</label><br>
    
    <input type="checkbox" id="mr" name="title" value="mr">
    <label for="mrs">Frau</label><br>
    
    <input type="checkbox" id="mrs" name="title" value="mrs">
    <label for="ms">Fräulein</label><br>
    
    <input type="checkbox" id="ms" name="title" value="ms">
    <label for="dr">Dr.</label><br>
    
    <input type="checkbox" id="dr" name="title" value="dr">
    <label for="prof">Prof.</label>
    
    <input type="checkbox" id="prof" name="title" value="prof">
</div>



"""
radios = """
<div>
    <label>Geschlecht:</label><br>
    <label for="male-radio">Männlich</label><br>
    
    <input type="radio" id="male-radio" name="gender" value="male">
    <label for="female-radio">Weiblich</label><br>
    
    <input type="radio" id="female-radio" name="gender" value="female">
    <label for="non-binary-radio">Non-binary</label><br>
    
    <input type="radio" id="non-binary-radio" name="gender" value="non-binary">
    <label for="prefer-not-to-say-radio">Keine Angabe</label>
    
    <input type="radio" id="prefer-not-to-say-radio" name="gender" value="prefer-not-to-say">
</div>

<div>
    <label>Sprache:</label><br>
    <label for="english-radio">Englisch</label><br>
    
    <input type="radio" id="english-radio" name="language" value="english">
    <label for="spanish-radio">Spanisch</label><br>
    
    <input type="radio" id="spanish-radio" name="language" value="spanish">
    <label for="french-radio">Französisch</label><br>
    
    <input type="radio" id="french-radio" name="language" value="french">
    <label for="chinese-radio">Chinesisch</label><br>
    
    <input type="radio" id="chinese-radio" name="language" value="chinese">
    <label for="hindi-radio">Hindi</label><br>
    
    <input type="radio" id="hindi-radio" name="language" value="hindi">
    <label for="arabic-radio">Arabisch</label><br>
    
    <input type="radio" id="arabic-radio" name="language" value="arabic">
    <label for="other-language-radio">Andere</label>
    
    <input type="radio" id="other-language-radio" name="language" value="other">
</div>

<div>
    <label>Familienstand:</label><br>
    <label for="single-radio">Ledig</label><br>
    
    <input type="radio" id="single-radio" name="marital-status" value="single">
    <label for="married-radio">Verheiratet</label><br>
    
    <input type="radio" id="married-radio" name="marital-status" value="married">
    <label for="divorced-radio">Geschieden</label><br>
    
    <input type="radio" id="divorced-radio" name="marital-status" value="divorced">
    <label for="widowed-radio">Verwitwet</label>
    
    <input type="radio" id="widowed-radio" name="marital-status" value="widowed">
</div>

<div>
    <label>Visumstatus:</label><br>
    <label for="tourist-radio">Touristenvisum</label><br>
    
    <input type="radio" id="tourist-radio" name="visa-status" value="tourist">
    <label for="student-radio">Studentenvisum</label><br>
    
    <input type="radio" id="student-radio" name="visa-status" value="student">
    <label for="work-radio">Arbeitsvisum</label><br>
    
    <input type="radio" id="work-radio" name="visa-status" value="work">
    <label for="resident-radio">Aufenthaltsvisum</label><br>
    
    <input type="radio" id="resident-radio" name="visa-status" value="resident">
    <label for="other-visa-radio">Andere</label>
    
    <input type="radio" id="other-visa-radio" name="visa-status" value="other">
</div>

<div>
    <label>Anrede:</label><br>
    <label for="mr-radio">Herr</label><br>
    
    <input type="radio" id="mr-radio" name="title" value="mr">
    <label for="mrs-radio">Frau</label><br>
    
    <input type="radio" id="mrs-radio" name="title" value="mrs">
    <label for="ms-radio">Fräulein</label><br>
    
    <input type="radio" id="ms-radio" name="title" value="ms">
    <label >Dr.</label><br>
    
    <input type="radio" id="dr-radio" name="title" value="dr">
</div>
"""




def get_selects(html,skip=False):
    data = []
    html = BeautifulSoup(html.lower(),'html.parser').find_all()
    for i,h in enumerate(html):
        h:PageElement
        options = h.find_all()
        
        if not skip:
            for j,o in enumerate(options):
                v = o.attrs.get('value')
                if v=="":
                    continue
                    pass
                
                #o['value']=j
                pass
            
        
        for j,o in enumerate(options):
            d={}
            v = o.attrs.get('value')
            v=str(v).strip()
            if v=="":
                continue
                pass
            if v==None:
                #o['value']=j
                pass
                
            
            d['element']=str(h)
            d['utterance'] = o.text
            d['label'] =j
            data.append(d)

    return data


d1 = get_selects(html)
#d2 = get_selects(html2)
#d3 = get_selects(html3)
#d4 = get_selects(y)
#d4 = get_selects(checks)
#c1= get_selects(checks)


#d1.extend(d2)
#d1.extend(d3)
#d1.extend(d4)

#df = pd.DataFrame.from_dict(d1)
#df
#df.to_excel('select.xlsx')
#df


,element,utterance,label
0,"<select name=""gender"">\n<option value="""">selec...",male,1
1,"<select name=""gender"">\n<option value="""">selec...",female,2
2,"<select name=""gender"">\n<option value="""">selec...",non-binary,3
3,"<select name=""gender"">\n<option value="""">selec...",prefer not to say,4
4,"<select name=""language"">\n<option value="""">sel...",english,1
...,...,...,...
2005,"<select aria-required=""false"" id=""job_applicat...",hispanic or latino,4
2006,"<select aria-required=""false"" id=""job_applicat...",white,5
2007,"<select aria-required=""false"" id=""job_applicat...",native hawaiian or other pacific islander,6
2008,"<select aria-required=""false"" id=""job_applicat...",two or more races,7


In [38]:
def print_df(df):
    for i,x in df.iterrows():
        print(x['query'])
        print(x['element'])
        print(x['utterance'])
        print(x['label'])
        
        print('------------------------------------------------')

<select name="gender">
<option value="">select gender</option>
<option value="male">male</option>
<option value="female">female</option>
<option value="non-binary">non-binary</option>
<option value="prefer-not-to-say">prefer not to say</option>
</select>
male
1
------------------------------------------------
<select name="gender">
<option value="">select gender</option>
<option value="male">male</option>
<option value="female">female</option>
<option value="non-binary">non-binary</option>
<option value="prefer-not-to-say">prefer not to say</option>
</select>
female
2
------------------------------------------------
<select name="gender">
<option value="">select gender</option>
<option value="male">male</option>
<option value="female">female</option>
<option value="non-binary">non-binary</option>
<option value="prefer-not-to-say">prefer not to say</option>
</select>
non-binary
3
------------------------------------------------
<select name="gender">
<option value="">select gender</opti

In [ ]:
import os
from bs4 import BeautifulSoup,PageElement,Comment
import pandas as pd


def preprocess_form(form):
    if isinstance(form,str):
        form = BeautifulSoup(form,'html.parser')
        
    form:PageElement
    comments=form.find_all(string=lambda text: isinstance(text, Comment))
    for comment in comments:
        comment.extract()
    all_tags = form.find_all()

    # Remove all tags except for 'link', 'input', and 'select' tags
    for i,element in enumerate(all_tags):
        element:PageElement
        element['nid']=i
        for k in ['style','class','bounding_box_rect','is_clickable']:
            if k in element.attrs:
                    del element[k]
                    pass
           
        if 'type' in element.attrs and element.attrs['type']=='hidden':
            element.unwrap()
            pass
            
        
        #if element.name  in ['div','text']: 
        if element.name not in ['a', 'input', 'select','radio','button','textarea','checkbox','option']:
            
            element.unwrap()
            #print(element)
            pass
        else:
            #element['backend_node_id']=i
            pass

            
                
            
    #print(form.prettify() )
    return form
    
import re

def get_query_element(f):
    query_element=[]
    mytext  = ''
    previous = ''
    after = ''
    texts = []
    
    for element in f:
        # Get the text before the element
        #print('-----------------------------')
        if element.name == None:
            t = element.text.replace('\n','').replace('\t','').strip()
            if t!='' and len(t)>2:
                #mytext+=t+'\n '
                texts.append(t)
        else:
            
            if element.attrs.get('type') in ['checkbox' ,'radio']:
                if len(query_element)>1:
                    if query_element[-1]['element'].find('type="checkbox"') != -1 or query_element[-1]['element'].find('type="radio"') != -1:
                        previous = query_element[-1]['previous']+query_element[-1]['query']+'\n'
                        pass
            #else: 
            # previous = texts[-2] if len(text)>1 else 0
                
            mytext += ''.join(texts[-1:]) +' '+ element.text.strip()
            mytext= mytext.strip()
            element : PageElement
            nid = element['nid']
            del element['nid']
            
            if len(query_element)>0:
                
                query_element[-1]['after']=texts[0] if len(texts)>0 else ''
            query_element.append({'query':mytext,'previous':previous,'after':'','element': str(element).replace('\n','').replace('  ','').strip(),'nid':nid})
            
            mytext =''
            texts=[]
        
    return query_element


def get_query_html(f):
    f:PageElement
    childs = f.find_all(string=True,recursive=False)
    for c in childs:
        print('---------------------')
        print(c)
    pass
    
    
    


In [4]:


            
def get_one_chat_template(person_str, query_element_str, label_str=None):
    d = [
        {   
            "role": "user",
            "content": """Your answer has to be in json format like {"answer":""}."""
            
            + person_str,
        },
        {
            "role": "user",
            "content": "What is this HTML element about?\n" + query_element_str,            
        },
    ]
    if label_str:
        d.append(
            {"role": "assistant", "content": '{"answer":"' + label_str + '"}'},
        )
        
    return d

        

with open('./html/1.html','r',encoding='utf-8') as f:
    html = f.read()
    
soup = BeautifulSoup(html,'html.parser')
forms = soup.find_all('form')
for f in forms:
    f = preprocess_form(f)
    qe = get_query_element(f)
    #print(qe)
 



NameError: name 'preprocess_form' is not defined

In [ ]:
def get_checkbox(html,skip=False):
    data = []
    html = BeautifulSoup(html.lower(),'html.parser').find_all()
    for i,h in enumerate(html):
        h:PageElement
        f = preprocess_form(h)
        qe = get_query_element(f)
        #print(qe)
        #options = h.find_all('div')
        #print(h)

        for d in qe:
            for v in qe:
                _l = 1 if v['query']==d['query'] else 0
                data.append({'query':d['query'],'previous':d['previous'],'after':d['after'], 'element':d['element'],'nid':d['nid'],'utterance':v['query'],'label':str(_l)})

    return data


r = get_checkbox(radios)
c = get_checkbox(checks)
r.extend(c)

In [ ]:
#y='\n'.join(h['element'].tolist())
r

In [ ]:
df = pd.DataFrame(r)
#df.to_excel('cheks.xlsx')

In [ ]:
#sel = pd.read_excel('select.xlsx',index_col=0)
#x = pd.concat([h,sel]).reset_index(drop=True)
#x.to_excel('query_element_8.xlsx',index=0)
#x

In [ ]:
t = pd.concat([train_dataset_2,df])
t.reset_index(drop=True)
t.to_excel('query_element_8.xlsx')
t

In [47]:
data = pd.read_excel('query_element_11.xlsx')


In [49]:
v = data['label'].value_counts()
d = v[v<5].to_dict()
for k in d.keys():
    df = data[data['label']==k]
    
    for i,x in df.iterrows():
        print(x['query'])
        print(x['element'])
        print(x['label'])
        print('------------------------------------------------')


Zeugnisse/Zertifikate
<input id="contentMap[2c93850264cf4fd20164f0ba5c6907e4].files[2c93850264cf4fd20164f0ba5c6907e4].file" name="contentMap[2c93850264cf4fd20164f0ba5c6907e4].files[2c93850264cf4fd20164f0ba5c6907e4].file" size="52" type="file" width="100%"/>
major
------------------------------------------------
Name der Qualifikation
<input autocomplete="on" dir="auto" id="9979-6-0" name="9979-6-0" placeholder="" tabindex="0" type="text" value=""/>
major
------------------------------------------------
Degree Select a Degree
<a href="javascript:void(0)" tabindex="-1"> Select a Degree </a>
major
------------------------------------------------
Discipline Select a Discipline
<a href="javascript:void(0)" tabindex="-1"> Select a Discipline </a>
major
------------------------------------------------
Zeugnis 2
<input data-check-oversize="1" id="167346" name="file_167346" tabindex="0" type="file"/>
transcript
------------------------------------------------
Zeugnis 3
<input data-check-oversiz

In [36]:
#c = data['element'].str.contains('<select')
#h=data.loc[c,:].reset_index(drop=True)
#h = BeautifulSoup(h[0],'html.parser').find_all('option')
#h.attrs.get('value')
for i,x in df.iterrows():
    #print(x['query'])
    print(x['element'])
    print(x['label'])
    print('------------------------------------------------')

<select name="gender">
<option value="">select gender</option>
<option value="male">male</option>
<option value="female">female</option>
<option value="non-binary">non-binary</option>
<option value="prefer-not-to-say">prefer not to say</option>
</select>
1
------------------------------------------------
<select name="gender">
<option value="">select gender</option>
<option value="male">male</option>
<option value="female">female</option>
<option value="non-binary">non-binary</option>
<option value="prefer-not-to-say">prefer not to say</option>
</select>
2
------------------------------------------------
<select name="gender">
<option value="">select gender</option>
<option value="male">male</option>
<option value="female">female</option>
<option value="non-binary">non-binary</option>
<option value="prefer-not-to-say">prefer not to say</option>
</select>
3
------------------------------------------------
<select name="gender">
<option value="">select gender</option>
<option value="male

In [25]:
y='\n'.join(h['element'].tolist())
y

'<select id="anrede" name="anrede"><option value="">Bitte auswählen</option><option value="Herr">Herr</option><option value="Frau">Frau</option><option value=" ">Divers</option><option value=" ">Keine Anrede</option></select>\n<select aria-required="true" id="c7b88c22-f4eb-ec11-bb3d-000d3adb0662" name="c7b88c22-f4eb-ec11-bb3d-000d3adb0662" required="required" title=""><option selected="selected" value="">Salutation</option><option value="919960100">Mr.</option><option value="919960200">Ms.</option><option value="919960300">Mx.</option></select>\n<select aria-required="true" id="c7b88c22-f4eb-ec11-bb3d-000d3adb0662" name="c7b88c22-f4eb-ec11-bb3d-000d3adb0662" required="required" title=""><option selected="selected" value="">Salutation</option><option value="919960100">Mr.</option><option value="919960200">Ms.</option><option value="919960300">Mx.</option></select>\n<select aria-required="true" id="50e2bc7d-1d67-ed11-9561-0022489e6486" name="50e2bc7d-1d67-ed11-9561-0022489e6486" required

In [26]:
s1 = get_selects(y)
s2 = get_selects(html)

[{'element': '<select id="anrede" name="anrede"><option value="">bitte auswählen</option><option value="herr">herr</option><option value="frau">frau</option><option value=" ">divers</option><option value=" ">keine anrede</option></select>',
  'utterance': 'herr',
  'label': 1},
 {'element': '<select id="anrede" name="anrede"><option value="">bitte auswählen</option><option value="herr">herr</option><option value="frau">frau</option><option value=" ">divers</option><option value=" ">keine anrede</option></select>',
  'utterance': 'frau',
  'label': 2},
 {'element': '<select aria-required="true" id="c7b88c22-f4eb-ec11-bb3d-000d3adb0662" name="c7b88c22-f4eb-ec11-bb3d-000d3adb0662" required="required" title=""><option selected="selected" value="">salutation</option><option value="919960100">mr.</option><option value="919960200">ms.</option><option value="919960300">mx.</option></select>',
  'utterance': 'mr.',
  'label': 1},
 {'element': '<select aria-required="true" id="c7b88c22-f4eb-ec11

In [43]:
df['query']=''
df

,element,utterance,label,query
0,"<select name=""gender"">\n<option value="""">selec...",male,1,
1,"<select name=""gender"">\n<option value="""">selec...",female,2,
2,"<select name=""gender"">\n<option value="""">selec...",non-binary,3,
3,"<select name=""gender"">\n<option value="""">selec...",prefer not to say,4,
4,"<select name=""language"">\n<option value="""">sel...",english,1,
...,...,...,...,...
2005,"<select aria-required=""false"" id=""job_applicat...",hispanic or latino,4,
2006,"<select aria-required=""false"" id=""job_applicat...",white,5,
2007,"<select aria-required=""false"" id=""job_applicat...",native hawaiian or other pacific islander,6,
2008,"<select aria-required=""false"" id=""job_applicat...",two or more races,7,


In [33]:
data2 = pd.read_excel('query_element_10.xlsx')
c2 = data2['element'].str.contains('<select')

h2=data2.loc[~c2,:].reset_index(drop=True)



,query,element,answer,label,utterance,previous,after,nid
0,Email Address,"<input id=""fieldEmail"" name=""cm-ukutgu-ukutgu""...",sharghi.majid@gmail.com,email,NaN,NaN,NaN,NaN
1,Please tick to confirm that you consent to us ...,"<a href=""/privacy-policy"" target=""_blank"" titl...",privacy,privacy,NaN,NaN,NaN,NaN
2,Subscribe,"<button type=""submit"">Subscribe</button>",subscribe,subscribe,NaN,NaN,NaN,NaN
3,Accept additional cookies,"<input id=""accept_all_cookies_button"" messaged...",cookie,cookie,NaN,NaN,NaN,NaN
4,Reject additional cookies,"<input id=""accept_essential_cookies_button"" me...",cookie,cookie,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1131,prof.,"<input id=""prof"" name=""title"" type=""checkbox"" ...",NaN,0,herr,frau\nfräulein\ndr.\n,NaN,15.0
1132,prof.,"<input id=""prof"" name=""title"" type=""checkbox"" ...",NaN,0,frau,frau\nfräulein\ndr.\n,NaN,15.0
1133,prof.,"<input id=""prof"" name=""title"" type=""checkbox"" ...",NaN,0,fräulein,frau\nfräulein\ndr.\n,NaN,15.0
1134,prof.,"<input id=""prof"" name=""title"" type=""checkbox"" ...",NaN,0,dr.,frau\nfräulein\ndr.\n,NaN,15.0


In [45]:
nnn= pd.concat([h2,df])
#nnn.to_excel('query_element_11.xlsx')


In [30]:
data= pd.read_excel('query_element_12.xlsx',index_col=0)
data['label'].value_counts().to_dict()

{2: 388,
 1: 375,
 3: 346,
 4: 298,
 5: 226,
 '0': 204,
 'search': 148,
 'cv': 144,
 6: 136,
 'submit': 123,
 7: 96,
 'email': 96,
 '1': 76,
 'first name': 66,
 'phone number': 47,
 'privacy': 46,
 'available from': 42,
 'password': 41,
 'address': 38,
 0: 34,
 'city': 33,
 8: 32,
 'login': 25,
 'linkedin': 23,
 9: 23,
 'find us': 19,
 'distance': 18,
 'salary': 17,
 'birth': 16,
 'cover letter': 15,
 'last name': 15,
 11: 14,
 12: 14,
 13: 14,
 10: 14,
 'job title': 12,
 'cookie': 11,
 'forgot password': 10,
 'captcha': 8,
 'country': 8,
 'major': 6,
 'zip': 5,
 'transcript': 4,
 'cancel': 4,
 'xing': 4,
 'pronoun': 3,
 'visa': 3,
 'skill': 3,
 'username': 3,
 'show password': 2,
 'graduation year': 2,
 'next': 2,
 'twitter': 2,
 'company': 2,
 'subscribe': 2,
 'ethnicity': 2,
 'newsletter': 2,
 'picture': 2,
 'recommender': 2,
 'university': 2,
 'github': 1,
 'civic number': 1,
 'years of experience': 1,
 'house number': 1,
 'English language level': 1}

In [22]:
c1 = data['element'].str.contains('<a')
c2 = data['element'].str.contains('<button')
c3 = data['element'].str.contains('type="button"')
c = c1|c2|c3


n = data.loc[~c,:].reset_index(drop=True)
h = data.loc[c,:].reset_index(drop=True)
m = h.copy()

"""
for i,x in h.iterrows():
    print(x['query'])
    print(x['element'])
    print(x['label'])
    print('------------------------------------------------')

"""

h['utterance']=h['query']
h['action']=1

m['utterance']="do not "+m['query']
m['action']=0
m

,query,element,answer,label,utterance,previous,after,nid,action
0,Please tick to confirm that you consent to us ...,"<a href=""/privacy-policy"" target=""_blank"" titl...",privacy,privacy,do not Please tick to confirm that you consent...,NaN,NaN,NaN,0
1,Subscribe,"<button type=""submit"">Subscribe</button>",subscribe,subscribe,do not Subscribe,NaN,NaN,NaN,0
2,Send,"<button data-ph-at-id=""send-button"" data-ph-id...",submit,submit,do not Send,NaN,NaN,NaN,0
3,Clear keyword search,"<button aria-label=""Clear keyword search"" type...",Google,search,do not Clear keyword search,NaN,NaN,NaN,0
4,Clear location search,"<button aria-label=""Clear location search"" typ...",search,search,do not Clear location search,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...
222,If you have any questions about the form conta...,"<a href=""mailto:joinus@isomorphiclabs.com"" tar...",www.linkedin.com,linkedin,do not If you have any questions about the for...,NaN,NaN,NaN,0
223,"Search By Business Area, Location or Language.","<a aria-controls=""moreOptionsDiv"" aria-expande...",search,search,"do not Search By Business Area, Location or La...",NaN,NaN,NaN,0
224,Clear Search Criteria,"<a href="""" role=""button"">Clear Search Criteria...",search,search,do not Clear Search Criteria,NaN,NaN,NaN,0
225,Activate,"<button aria-label=""Submit get notified for si...",submit,submit,do not Activate,NaN,NaN,NaN,0


In [28]:
x=pd.concat([m,h])
nn = pd.concat([n,x])
nn= nn.reset_index(drop=True)
#nn.to_excel('query_element_12.xlsx')
